In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib

In [2]:
# MEAN
speaker_profile = pd.read_csv('result\\voice_recognition\\training_mean\\vr_speaker_mean_features.csv')

In [3]:
speaker_profile

,file_name,mfcc_feature1,mfcc_feature2,mfcc_feature3,mfcc_feature4,mfcc_feature5,mfcc_feature6,mfcc_feature7,mfcc_feature8,mfcc_feature9,...,mfcc_feature34,mfcc_feature35,mfcc_feature36,mfcc_feature37,mfcc_feature38,mfcc_feature39,MeanRMS,MeanZCR,MeanSpectralCentroid,MeanSpectralBandwidth
0,Recording1.mp3,-466.47586,113.188000,-31.554749,58.069960,12.549055,11.572303,-0.637779,-0.476538,-1.047442,...,-0.025053,-0.021559,-0.014921,-0.020318,-0.024983,-0.006847,0.016590,0.091200,1477.526239,1285.109936
1,Recording10.mp3,-462.36844,75.371730,-22.990372,55.777367,7.616342,18.860722,-5.601764,1.031670,0.352627,...,0.020604,0.006262,-0.013555,-0.008639,-0.011821,0.003830,0.017908,0.124919,1868.689684,1427.695075
2,Recording2.mp3,-494.67728,101.057550,-27.088408,46.063230,12.271044,8.745768,-2.987978,-2.950048,0.919167,...,-0.003895,-0.001905,-0.012755,-0.004770,-0.000305,0.013672,0.020544,0.077840,1396.769670,1322.673459
3,Recording3.mp3,-483.11258,103.349540,-18.448746,43.987610,11.716349,8.919379,-2.134769,0.029024,2.959834,...,-0.001163,-0.004468,-0.005017,0.002327,-0.000646,0.000129,0.019035,0.100713,1558.875741,1381.989533
4,Recording4.mp3,-467.58750,115.525540,-26.715477,44.850174,10.586900,9.302888,-3.408906,-1.295586,0.420204,...,-0.001237,0.017196,0.015745,0.000190,-0.009750,0.003944,0.017176,0.090676,1513.490433,1380.374358
5,Recording5.mp3,-468.93192,95.931120,-22.821474,35.816044,1.117845,4.841961,-2.747456,2.456876,-1.691102,...,0.001675,0.001273,0.002474,0.008833,0.000116,-0.001845,0.020141,0.104348,1592.569822,1468.954597
6,Recording6.mp3,-434.86807,112.130295,-22.810982,53.075650,12.100018,8.637113,-6.769464,1.086398,-2.606873,...,0.001338,0.007670,0.002044,0.001143,-0.006781,-0.008694,0.021101,0.083437,1493.441258,1430.868262
7,Recording7.mp3,-456.61258,101.399530,-14.078453,49.710340,11.470749,5.569509,-2.300609,0.427971,2.139126,...,-0.002370,-0.000991,-0.010611,-0.014083,-0.011373,0.000653,0.019141,0.080919,1378.013182,1378.983510
8,Recording8.mp3,-475.19067,100.311050,-12.050959,45.496830,11.391419,5.529108,-3.854567,-0.944330,-0.956235,...,-0.020397,-0.015603,-0.010603,-0.000347,-0.008924,0.004940,0.018455,0.093311,1556.366723,1455.661085
9,Recording9.mp3,-476.58188,94.279630,-26.243538,41.434730,5.930160,8.973958,-5.152748,-3.119601,-1.307506,...,-0.002584,0.008839,-0.007553,-0.000823,0.004018,-0.005649,0.024441,0.087714,1468.671421,1401.201102


In [4]:
X = speaker_profile.drop(columns="file_name").values

In [5]:
# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))  # 3D input (batch_size, num_features, 1)
joblib.dump(scaler, 'vr_scaler_3d.pkl')

['vr_scaler_3d.pkl']

In [26]:
len(X_train[2])

43

In [7]:
y_train = np.ones((X_train.shape[0],))  # Labels for the subject's voice

In [8]:
y_train

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [33]:
X_train.shape[1]

43

In [42]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input

# Get the number of features from X_train
num_features = X_train.shape[1]  # This should be the number of features (columns) in X_train
num_timesteps = 1  # Since we're treating it as 1D time series-like input

# Build the model (with correct input/output shape matching)
speaker_identification_model = Sequential([
    # Convolutional Layers
    Conv2D(16, (2, 2), activation='relu', input_shape=(num_features, 1, 1), padding='same'),
    MaxPooling2D((2, 1)),  # Pooling Layer
    
    Conv2D(32, (2, 2), activation='relu', padding='same'),
    MaxPooling2D((2, 1)),  # Pooling Layer
    
    # Flatten Layer: Convert the feature maps into a 1D vector
    Flatten(),  # Ensure this flattens the data
    
    # Hidden Layer: Extract and transform features from the flattened vector
    Dense(64, activation='relu'),  # The input for this layer is now a flattened 1D vector
    Dropout(0.5),
    
    # Output Layer: Prediction for one-class classification (Reconstruction output)
    Dense(num_features, activation='linear')  # Output size should match the original input size
])

# Compile the model with MSE loss
speaker_identification_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse'])

# Display the model summary
speaker_identification_model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_14 (Conv2D)                   │ (None, 43, 1, 16)           │              80 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_14 (MaxPooling2D)      │ (None, 21, 1, 16)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_15 (Conv2D)                   │ (None, 21, 1, 32)           │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_15 (MaxPooling2D)      │ (None, 10, 1, 32)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_7 (Flatten)                  │ (None, 320)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 64)                  │          20,544 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ (None, 43)                  │           2,795 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 25,499 (99.61 KB)

 Trainable params: 25,499 (99.61 KB)

 Non-trainable params: 0 (0.00 B)

In [43]:
history = speaker_identification_model.fit(X_train, X_train, epochs=20, batch_size=16, validation_split=0.2)

Epoch 1/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - loss: 1.1275 - mse: 1.1275 - val_loss: 0.7095 - val_mse: 0.7095
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - loss: 1.1077 - mse: 1.1077 - val_loss: 0.7138 - val_mse: 0.7138
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - loss: 1.0693 - mse: 1.0693 - val_loss: 0.7162 - val_mse: 0.7162
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - loss: 1.0642 - mse: 1.0642 - val_loss: 0.7180 - val_mse: 0.7180
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step - loss: 1.0536 - mse: 1.0536 - val_loss: 0.7204 - val_mse: 0.7204
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - loss: 1.0539 - mse: 1.0539 - val_loss: 0.7225 - val_mse: 0.7225
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - loss: 1.0404 - mse: 1.0404 - val_loss: 0.7243 - val_mse: 0.7243
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - loss: 1.0310 - mse: 1.0310 - val_loss: 0.7269 - val_mse: 0.7269
Epoch 9/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - loss: 1.0225 - mse: 1.0225 - va

In [141]:
# TEST for mean

# single_mfcc = pd.read_csv('result\\voice_recognition\\test_mean\\vr_same_mean_test1.csv') #same
# single_mfcc = pd.read_csv('result\\voice_recognition\\test_mean\\vr_same_mean_test2.csv') # same
# single_mfcc = pd.read_csv('result\\voice_recognition\\test_mean\\vr_same_mean_test3.csv') # same
# single_mfcc = pd.read_csv('result\\voice_recognition\\test_mean\\vr_same_mean_test4.csv') #same
# single_mfcc = pd.read_csv('result\\voice_recognition\\test_mean\\vr_notsame_mean_test1.csv') #not same
# single_mfcc = pd.read_csv('result\\voice_recognition\\test_mean\\vr_notsame_mean_test2.csv') #not same
single_mfcc = pd.read_csv('result\\voice_recognition\\test_mean\\vr_notsame_mean_test3.csv') #not same

scaler = joblib.load('vr_scaler_3d.pkl')

# Drop unnecessary columns and scale the data
single_mfcc = single_mfcc.drop(columns=['file_name']).values
single_scaled = scaler.transform(single_mfcc)

# Reshape the input to match the CNN model's expected input shape
single_scaled = single_scaled.reshape(1, num_features, 1, 1)  # Reshaping for 2D Conv

In [142]:
single_reshaped.shape
# num_features

(43, 1, 1)

In [144]:
# Get the reconstructed audio features from the model
reconstructed_audio = speaker_identification_model.predict(single_scaled)
print("Original: ", single_scaled.flatten())
print("Reconstructed: ", reconstructed_audio.flatten())
# Reshape the predicted output to match the original input format for comparison
reconstructed_audio_flat = reconstructed_audio.reshape(num_features)

# Calculate the reconstruction error (mean absolute error or mean squared error)
reconstruction_error = np.mean(np.abs(single_mfcc.flatten() - reconstructed_audio_flat))

# Calculate reconstruction error for the training data
training_reconstruction_errors = np.mean(np.abs(X_train - speaker_identification_model.predict(X_train)), axis=1)
print("Reconstruction errors for training data: ", training_reconstruction_errors)

# Set the threshold as mean + 1 or 2 standard deviations (fine-tune this)
threshold = np.mean(training_reconstruction_errors) + np.std(training_reconstruction_errors)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Original:  [10.87898738  2.30260492  2.86354979 -1.22479159  0.91535231 -1.56832391
  0.94661564  1.25958983 -7.20149438  0.81015573  1.73117024 -0.63735724
 -0.05795852  4.52047171  6.35277673  1.85826675  2.32195163  2.53121506
  0.31991199  3.0939646   1.79277585  0.40175141  1.65094881  1.40815947
  1.60478461  0.75197829  2.49634183  0.86031686 -0.87130821 -0.02809237
  0.74873298 -0.20484016  0.7277862   0.3541109   0.0260901   0.16836244
 -0.50031591  0.39456864  0.31966797 23.04943231 -1.79572878 -0.07579233
  7.54024039]
Reconstructed:  [-0.03947283  0.19783603 -0.5484882   1.131783   -0.10866912  1.0071304
 -1.0489668   0.5962452   0.563318    0.93049717 -0.18138239  0.68299115
 -0.22916996  0.33512396 -0.38483953 -0.3957238   0.11352189 -0.23994207
 -0.6672748   0.24087223 -0.20845832 -0.16319744 -0.02244191  0.29863748
 -1.2520156   0.55102885 -0.12849136  0.01401274 -0.0921421   0.6553756
  1.2900505   0.52159894  0.04612526  0.7874846

ValueError: Exception encountered when calling Sequential.call().

[1mCannot take the length of shape with unknown rank.[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=<unknown>, dtype=float32)
  • training=False
  • mask=None

In [140]:
# Classify whether the test audio belongs to the speaker or not based on the error
print(reconstruction_error)
if reconstruction_error < threshold:
    print("Speaker Identified (Same speaker)")
else:
    print("Different Speaker Detected")

89.7102369375567
Different Speaker Detected
